In [2]:
from get_dataset import get_zhao_dataset

In [11]:
import json
data = get_zhao_dataset('dd')
with open("saver_topk.jsonl",'r') as f:
    prediction_data = [json.loads(el) for el in f.readlines()]
    prediction_data = [{'human':el['human_score'],'pred':el['original_pred']} for el in prediction_data]

{'corpus': 'dd',
 'ctx': ['sure . where do you want to go ? are you in the mood for anything in '
         'particular ?',
         "how about some dumplings ? i just ca n't get enough of them .",
         "dumplings again ? ! oh , amy , let 's try something new !",
         'well , what do you have in mind ?',
         'how about the mongolian hot pot ?'],
 'human_score': 1.75,
 'hyp': 'did you go to school today ?',
 'model': 'negative-sample',
 'ref': "oh , it 's too spicy for me . do n't you remember last time when i "
        'tasted the lamb ? it was so spicy my eyes teared up !'}


In [28]:
counter = []
for item_idx,item in enumerate(data):
    assert item['human_score'] == prediction_data[item_idx]['human']
    human_score = (item['human_score']-1)/4
    assert 0<=human_score<=1
    prediction = round(prediction_data[item_idx]['pred'],2)
    if human_score<0.5 and prediction>0.6:
        print(item_idx)
        counter.append(True)
        for idx,c in enumerate(item['ctx']):
            print("C{}: ".format(idx),c)
        print()
        print("G: ",item['ref'])
        print('H: ',item['hyp'])
        print("Human: ",human_score)
        print("Model: ",prediction)
        print()
    else:
        counter.append(False)
print("False Positive: {}".format(sum(counter)/len(counter)))

2
C0:  well , how does it look ?
C1:  it 's a perfect fit .
C2:  let me pay for it now .

G:  cash , credit card , or debit card ?
H:  i 'm so sorry . i forgot to fill out the form .
Human:  0.3125
Model:  0.78

10
C0:  can i help you , sir ?
C1:  hello , may i make an appointment for sometime early this afternoon ?
C2:  i 'm sorry . all the appointments are filled .

G:  oh , what a pity . i 'd like to see the doctor as soon as possible . i have a sharp pain in my chest .
H:  but could you make it clear that i 'm in the airport for lunch ?
Human:  0.1875
Model:  0.92

19
C0:  i have a problem with my cable .
C1:  what about it ?
C2:  my cable has been out for the past week or so .
C3:  the cable is down right now . i am very sorry .
C4:  when will it be working again ?

G:  it should be back on in the next couple of days .
H:  this last .
Human:  0.25
Model:  0.97

27
C0:  did you get your bus pass ?
C1:  i have n't gone to get it yet .
C2:  why have n't you got it ?
C3:  i do n't kno

In [2]:
from transformers import BartForConditionalGeneration, BartTokenizer
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large", force_bos_token_to_be_generated=True)
tok = BartTokenizer.from_pretrained("facebook/bart-large")
example_english_phrase = "UN Chief Says There Is No <mask> in Syria"
batch = tok(example_english_phrase, return_tensors='pt')
generated_ids = model.generate(batch['input_ids'])
assert tok.batch_decode(generated_ids, skip_special_tokens=True) == ['UN Chief Says There Is No Plan to Stop Chemical Weapons in Syria']

In [22]:
example_english_phrase ="oh , it 's not just me . do n't you remember last time when <mask> ?"
batch = tok(example_english_phrase, return_tensors='pt')
generated_ids = model.generate(batch['input_ids'],max_length=300)#,do_sample=True,top_k=100,num_return_sequences=10)
decoded = tok.batch_decode(generated_ids, skip_special_tokens=True)
print(decoded)

["oh, it's not just me. don't you remember last time when I was in the hospital?"]


In [45]:
print(tok.convert_tokens_to_ids(['.',',','!','..','...']))
for i in range(10):
    print(tok.convert_ids_to_tokens([479,1275]))

[4, 6, 328, 7586, 734]
['Ġ.', 'Ġred']
['Ġ.', 'Ġred']
['Ġ.', 'Ġred']
['Ġ.', 'Ġred']
['Ġ.', 'Ġred']
['Ġ.', 'Ġred']
['Ġ.', 'Ġred']
['Ġ.', 'Ġred']
['Ġ.', 'Ġred']
['Ġ.', 'Ġred']


In [46]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
TreebankWordDetokenizer().detokenize(" no , not on the train , but in the station .".strip().split())
from transformers import BertTokenizer
berttok = BertTokenizer.from_pretrained("bert-base-uncased")
berttok.tokenize("no , not me.")

['no', ',', 'not', 'me', '.']

In [52]:
def get_usencoder():
    import tensorflow_hub as hub

    return hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
encoder = get_usencoder()
a = encoder(['i like this'])


In [53]:
a

<tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[ 4.13115956e-02, -5.58152329e-03,  3.13844979e-02,
         5.64465970e-02,  7.35069811e-02,  2.03321334e-02,
         9.77036729e-03,  6.13079444e-02,  7.10977316e-02,
         7.40062445e-02,  4.68988344e-02, -4.43977267e-02,
        -9.84571502e-03,  1.02056593e-01,  1.23268990e-02,
         4.75421511e-02,  1.16306394e-02, -1.59528886e-03,
        -4.57843486e-03, -4.12269793e-02, -4.54492979e-02,
        -8.57749116e-03,  4.05394472e-02,  5.54446802e-02,
        -3.92194763e-02, -2.37115957e-02, -2.99052317e-02,
        -3.12393121e-02,  1.60388779e-02, -6.61575422e-02,
        -2.40907222e-02,  1.47254206e-02, -2.40781363e-02,
        -1.78729594e-02, -1.31732281e-02,  2.99148783e-02,
         5.56203024e-03,  5.47429994e-02, -5.94272576e-02,
        -1.42630599e-02, -2.03540158e-02,  2.16410477e-02,
         2.40427833e-02, -4.37166803e-02, -3.91214415e-02,
         6.56230077e-02, -8.43839999e-03,  4.47384492e-02,
      